In [2]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle
import tensorflow as tp
from collections import Counter




ModuleNotFoundError: No module named 'librosa'

In [ ]:
# Define raw audio dictionary
raw_audio = {}

# Loop through directories and label audio files
directories = ['hungry', 'belly_pain', 'burping', 'discomfort', 'tired']
for directory in directories:
    path = '/content/drive/MyDrive/3rd year projects/Research/Thesis/Data and affecting factors/Data Source/donateacry_corpus_cleaned_and_updated_data/' + directory
    for filename in os.listdir(path):
        if filename.endswith(".wav"):
            raw_audio[os.path.join(path, filename)] = directory


In [ ]:
# Define function to extract MFCC features and chop audio
def extract_mfcc(audio_file, max_length=100):
    audiofile, sr = librosa.load(audio_file)
    fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=20)
    if fingerprint.shape[1] < max_length:
        pad_width = max_length - fingerprint.shape[1]
        fingerprint_padded = np.pad(fingerprint, pad_width=((0, 0), (0, pad_width)), mode='constant')
        return fingerprint_padded.T
    elif fingerprint.shape[1] > max_length:
        return fingerprint[:, :max_length].T
    else:
        return fingerprint.T

# Chop audio and extract MFCC features for each track
X = []
y = []
max_length = 100
for i, (audio_file, label) in enumerate(raw_audio.items()):
    mfcc_features = extract_mfcc(audio_file, max_length=max_length)
    X.append(mfcc_features)
    y.append(label)



In [ ]:
# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Flatten the features and labels
X_flat = X.reshape(X.shape[0], -1)
y_flat = y

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flat, y_flat, test_size=0.2, random_state=42)


In [ ]:

# Train and evaluate models
models = [
    ('Random Forest', RandomForestClassifier(n_estimators=25, max_features=5)),
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('SVM', SVC()),
]



In [ ]:
print("Model, Accuracy, Precision, Recall")
for model_name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    print(f"{model_name}: {accuracy}, {precision}, {recall}")

Model, Accuracy, Precision, Recall
Random Forest: 0.7934782608695652, 0.6296077504725898, 0.7934782608695652


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: 0.717391304347826, 0.6434694238246731, 0.717391304347826
Decision Tree: 0.6847826086956522, 0.6806534795665231, 0.6847826086956522
SVM: 0.7934782608695652, 0.6296077504725898, 0.7934782608695652


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(X_train.shape)


(364, 2000)


In [ ]:
# Reshape data for LSTM input
n_samples, n_features = X_train.shape[0], X_train.shape[1] // 100
n_timesteps = 100
X_train_lstm = X_train.reshape((n_samples, 100, 20))
n_samples_test = X_test.shape[0]
X_test_lstm = X_test.reshape((n_samples_test, n_timesteps, n_features))

# Convert labels to numeric values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define LSTM model
lstm_model = Sequential([
    LSTM(units=128, input_shape=(n_timesteps, n_features)),
    Dropout(0.2),
    Dense(units=64, activation='relu'),
    Dropout(0.2),
    Dense(units=len(np.unique(y_train_encoded)), activation='softmax')
])


# Compile LSTM model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train LSTM model
lstm_model.fit(X_train_lstm, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate LSTM model
_, accuracy = lstm_model.evaluate(X_test_lstm, y_test_encoded)
print("Accuracy:", accuracy)

from sklearn.metrics import precision_score, recall_score

# Predict probabilities for the test dataset using the trained LSTM model
predicted_probabilities = lstm_model.predict(X_test_lstm)

# Convert probabilities to class labels
predicted_labels = np.argmax(predicted_probabilities, axis=1)

# Calculate precision
precision = precision_score(y_test_encoded, predicted_labels, average='weighted')

# Calculate recall
recall = recall_score(y_test_encoded, predicted_labels, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

Epoch 1/10
10/10 [==============================] - 7s 178ms/step - loss: 1.1529 - accuracy: 0.6014 - val_loss: 0.6081 - val_accuracy: 0.8630
Epoch 2/10
10/10 [==============================] - 1s 121ms/step - loss: 0.6935 - accuracy: 0.8454 - val_loss: 0.6093 - val_accuracy: 0.8630
Epoch 3/10
10/10 [==============================] - 1s 116ms/step - loss: 0.6518 - accuracy: 0.8454 - val_loss: 0.6019 - val_accuracy: 0.8630
Epoch 4/10
10/10 [==============================] - 1s 117ms/step - loss: 0.6518 - accuracy: 0.8454 - val_loss: 0.5967 - val_accuracy: 0.8630
Epoch 5/10
10/10 [==============================] - 1s 122ms/step - loss: 0.6063 - accuracy: 0.8454 - val_loss: 0.5909 - val_accuracy: 0.8630
Epoch 6/10
10/10 [==============================] - 2s 158ms/step - loss: 0.6113 - accuracy: 0.8454 - val_loss: 0.5849 - val_accuracy: 0.8630
Epoch 7/10
10/10 [==============================] - 2s 229ms/step - loss: 0.6290 - accuracy: 0.8454 - val_loss: 0.5843 - val_accuracy: 0.8630
Epoch 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import joblib

# Save the model
joblib.dump(lstm_model, "lstm_audio_model.joblib")


['lstm_audio_model.joblib']

In [ ]:
def pickle_model(model, modelname):
    directory = 'models'
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(os.path.join(directory, str(modelname) + '.pkl'), 'wb') as f:
        return pickle.dump(model, f)

model = RandomForestClassifier()
model.fit(X_train, y_train)
pickle_model(model, "myRandomForest")

In [ ]:
def getModel(pickle_path):
  with open(pickle_path, 'rb') as f:
        return pickle.load(f)

In [ ]:
import joblib
model_path = "models/myRandomForest.pkl"  # Replace with your model path
with open(model_path, "rb") as f:
    model = joblib.load(f)

In [ ]:
pip install pydub

In [ ]:
from io import BytesIO
from pydub import AudioSegment
import wave
import math
import os
import uuid

# Define the function to chop the audio
def chop_new_audio(audio_data, folder):
    os.makedirs(folder, exist_ok=True)  # Create directory if it doesn't exist
    audio = wave.open(audio_data, 'rb')
    frame_rate = audio.getframerate()
    n_frames = audio.getnframes()
    window_size = 2 * frame_rate
    num_secs = int(math.ceil(n_frames / frame_rate))
    last_number_frames = 0
    for i in range(num_secs):
        shortfilename = str(uuid.uuid4())  # Generate a unique filename
        snippetfilename = f"{folder}/{shortfilename}snippet{i+1}.wav"
        snippet = wave.open(snippetfilename, 'wb')
        snippet.setnchannels(2)
        snippet.setsampwidth(audio.getsampwidth())
        snippet.setframerate(frame_rate)
        snippet.setnframes(audio.getnframes())
        snippet.writeframes(audio.readframes(window_size))
        audio.setpos(audio.tell() - 1 * frame_rate)

         # Check if the frame size of the snippet matches the previous snippets
        if last_number_frames < 1:
            last_number_frames = snippet.getnframes()
        elif snippet.getnframes() != last_number_frames:
            os.rename(snippetfilename, f"{snippetfilename}.bak")
        snippet.close()

# Example usage
if __name__ == "__main__":
    # Example of reading audio data from a file-like object (e.g., uploaded file)
    with open('hungry baby.mp3', 'rb') as f:
        mp3_data = f.read()

    audio = AudioSegment.from_mp3(BytesIO(mp3_data))
    wav_data = BytesIO()
    audio.export(wav_data, format="wav")
    wav_data.seek(0)

    folder_name = "samples"
    chop_new_audio(wav_data, folder_name)
    print("Audio chopped successfully!")


Audio chopped successfully!


In [ ]:
# Predict on new audio snippets
predictions = []

folder_path = 'samples/'

for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        audiofile, sr = librosa.load(os.path.join(folder_path, filename))
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=20)
        fingerprint_flat = fingerprint.reshape(-1)  # Flatten the MFCC features
        # Pad or truncate features to match the number of features used for training
        if len(fingerprint_flat) < 2000:
            fingerprint_flat = np.pad(fingerprint_flat, (0, 2000 - len(fingerprint_flat)))
        elif len(fingerprint_flat) > 2000:
            fingerprint_flat = fingerprint_flat[:2000]
        prediction = model.predict([fingerprint_flat])  # Reshape to match expected input format
        predictions.append(prediction[0])

from collections import Counter
data = Counter(predictions)
print(data.most_common())  # Returns all unique items and their counts
print(data.most_common(1))  # Returns the most common prediction


[('discomfort', 35), ('tired', 27), ('hungry', 1)]
[('discomfort', 35)]
